## Deliverable 2. Create a Customer Travel Destinations Map.

In [25]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np
# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [26]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Mercedes,-34.6515,-59.4307,55.00,94,97,4.41,overcast clouds
1,1,Rikitea,-23.1203,-134.9692,77.25,77,27,6.02,light rain
2,2,Talnakh,69.4865,88.3972,12.61,85,98,9.37,overcast clouds
3,3,Hobart,-42.8794,147.3294,62.60,45,20,19.57,few clouds
4,4,East London,-33.0153,27.9116,64.40,63,0,5.75,clear sky


In [27]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [29]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]

In [33]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                165
City                   165
Lat                    165
Lng                    165
Max Temp               165
Humidity               165
Cloudiness             165
Wind Speed             165
Current Description    165
dtype: int64

In [32]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                           (preferred_cities_df["Max Temp"] >= min_temp)].dropna()

In [35]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Rikitea,77.25,light rain,-23.1203,-134.9692,
10,Pangkalanbuun,86.59,light rain,-2.6833,111.6167,
14,Atuona,80.24,broken clouds,-9.8000,-139.0333,
15,Vaini,78.80,light rain,-21.2000,-175.2000,
16,Lorengau,83.80,broken clouds,-2.0226,147.2712,
27,Saint-Philippe,82.40,light rain,-21.3585,55.7679,
29,Avarua,77.00,broken clouds,-21.2078,-159.7750,
36,Nishihara,82.40,scattered clouds,26.1842,127.7558,
43,Sassandra,75.33,overcast clouds,4.9500,-6.0833,
47,Malakal,77.86,overcast clouds,9.5334,31.6605,


In [51]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
            hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
            print("Hotel not found... skipping.")
            hotel_df.loc[index, "Hotel Name"] = pd.NaT
            pass
        
hotel_df.head(50)

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


,City,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Rikitea,77.25,light rain,-23.1203,-134.9692,Pension Maro'i
10,Pangkalanbuun,86.59,light rain,-2.6833,111.6167,Grand Kecubung Hotel
14,Atuona,80.24,broken clouds,-9.8000,-139.0333,Villa Enata
15,Vaini,78.80,light rain,-21.2000,-175.2000,Keleti Beach Resort
16,Lorengau,83.80,broken clouds,-2.0226,147.2712,Lorengau Harbourside Hotel
27,Saint-Philippe,82.40,light rain,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""
29,Avarua,77.00,broken clouds,-21.2078,-159.7750,Paradise Inn
36,Nishihara,82.40,scattered clouds,26.1842,127.7558,Minshuku Agaihama
43,Sassandra,75.33,overcast clouds,4.9500,-6.0833,la terrasse
47,Malakal,77.86,overcast clouds,9.5334,31.6605,Nile Palace Hotel


In [53]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna(subset = ["Hotel Name"])
clean_hotel_df.head(50)

,City,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Rikitea,77.25,light rain,-23.1203,-134.9692,Pension Maro'i
10,Pangkalanbuun,86.59,light rain,-2.6833,111.6167,Grand Kecubung Hotel
14,Atuona,80.24,broken clouds,-9.8000,-139.0333,Villa Enata
15,Vaini,78.80,light rain,-21.2000,-175.2000,Keleti Beach Resort
16,Lorengau,83.80,broken clouds,-2.0226,147.2712,Lorengau Harbourside Hotel
27,Saint-Philippe,82.40,light rain,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""
29,Avarua,77.00,broken clouds,-21.2078,-159.7750,Paradise Inn
36,Nishihara,82.40,scattered clouds,26.1842,127.7558,Minshuku Agaihama
43,Sassandra,75.33,overcast clouds,4.9500,-6.0833,la terrasse
47,Malakal,77.86,overcast clouds,9.5334,31.6605,Nile Palace Hotel


In [54]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [57]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Current Weather</dt><dd>{Current Description}
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [59]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations,info_box_content=hotel_info)
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))